# 7.Python: 대리점 데이터를 가공하는 테크닉10 (후기)

### 목표: 지저분한 데이터 처리하기

###  데이터 읽기

In [1]:
import pandas as pd
uriage_data = pd.read_csv("/Users/icda/pythondata/pyda100-master/2장/uriage.csv")
uriage_data.head()

,purchase_date,item_name,item_price,customer_name
0,2019-06-13 18:02,상품A,100.0,김가온
1,2019-07-13 13:05,상 품 S,NaN,김우찬
2,2019-05-11 19:42,상 품 a,NaN,김유찬
3,2019-02-12 23:40,상품Z,2600.0,김재현
4,2019-04-22 3:09,상품a,NaN,김강현


In [2]:
# read_excel 설치하기
# !pip install openpyxl

In [3]:
# xlsx 파일 읽기
kokyaku_data = pd.read_excel("/Users/icda/pythondata/pyda100-master/2장/kokyaku_daicho.xlsx")
kokyaku_data.head()

,고객이름,지역,등록일
0,김 현성,H시,2018-01-04 00:00:00
1,김 도윤,E시,42782
2,김 지한,A시,2018-01-07 00:00:00
3,김 하윤,F시,42872
4,김 시온,E시,43127


* item_name 과 item_price 결측오류가 있음
* item_name 상품 이름 형식이 통일 안되어있으 / item_price 결측치 있음
* 등록일 서식이 통일 안되어있음

-> 데이터 정합성에 문제가 있다

### 데이터 오류 살펴보기

In [4]:
# 매출이력 데이터 오류 확인
uriage_data["item_name"].head()

0      상품A
1    상 품 S
2    상 품 a
3      상품Z
4      상품a
Name: item_name, dtype: object

In [5]:
uriage_data["item_price"].head()

0     100.0
1       NaN
2       NaN
3    2600.0
4       NaN
Name: item_price, dtype: float64

### 데이터 오류가 있는 상태로 집계

In [6]:
uriage_data["purchase_data"] = pd.to_datetime(uriage_data["purchase_date"])
uriage_data["purchase_month"] = uriage_data["purchase_data"].dt.strftime("%Y%m")
res = uriage_data.pivot_table(index="purchase_month", columns="item_name",
                              aggfunc="size", fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1,0,0,0,0,0,0,0,0,0,...,1,1,1,0,0,0,0,0,0,0
201902,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,1,1,0,0
201903,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
201904,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,0,0,0,0
201905,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1
201906,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0
201907,0,0,0,0,0,0,0,0,1,0,...,0,0,1,0,2,0,0,0,0,0


In [7]:
res = uriage_data.pivot_table(index="purchase_month", columns="item_name", values="item_price",
                              aggfunc="sum", fill_value=0)
res

item_name,상 품 n,상품 E,상품 M,상품 P,상품 S,상품 W,상품 X,상품W,상 품O,상 품Q,...,상품k,상품l,상품o,상품p,상품r,상품s,상품t,상품v,상품x,상품y
purchase_month,,,,,,,,,,,,,,,,,,,,,
201901,1400.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1100.0,1200.0,1500.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201902,0.0,0.0,0.0,0.0,0.0,0.0,2400.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,2000.0,2200.0,0.0,0.0
201903,0.0,500.0,1300.0,1600.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
201904,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,1700.0,...,0.0,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0
201905,0.0,0.0,0.0,0.0,1900.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2500.0
201906,0.0,0.0,0.0,0.0,0.0,2300.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1600.0,0.0,0.0,0.0,0.0,2400.0,0.0
201907,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1500.0,0.0,1800.0,0.0,0.0,0.0,0.0,0.0


### 상품명 오류 수정하기

In [8]:
#item_name 데이터 전체 건수 확인(원래는 A~Z까지 26개)
print(len(pd.unique(uriage_data.item_name)))

99


In [9]:
####데이터 오류 수정
uriage_data["item_name"] = uriage_data["item_name"].str.upper()
uriage_data["item_name"] = uriage_data["item_name"].str.replace(" ","")
uriage_data["item_name"] = uriage_data["item_name"].str.replace("  ","")
uriage_data.sort_values(by=["item_name"], ascending=True)

,purchase_date,item_name,item_price,customer_name,purchase_data,purchase_month
0,2019-06-13 18:02,상품A,100.0,김가온,2019-06-13 18:02:00,201906
1748,2019-05-19 20:22,상품A,100.0,김시훈,2019-05-19 20:22:00,201905
223,2019-06-25 8:13,상품A,100.0,김유진,2019-06-25 08:13:00,201906
1742,2019-06-13 16:03,상품A,100.0,김건희,2019-06-13 16:03:00,201906
1738,2019-02-10 0:28,상품A,100.0,김하랑,2019-02-10 00:28:00,201902
...,...,...,...,...,...,...
2880,2019-04-22 0:36,상품Y,NaN,김동욱,2019-04-22 00:36:00,201904
2881,2019-04-30 14:21,상품Y,NaN,김하준,2019-04-30 14:21:00,201904
1525,2019-01-24 10:27,상품Y,2500.0,김범준,2019-01-24 10:27:00,201901
1361,2019-05-28 13:45,상품Y,2500.0,김수현,2019-05-28 13:45:00,201905


In [10]:
print(len(pd.unique(uriage_data["item_name"])))
print(pd.unique(uriage_data["item_name"]))

26
['상품A' '상품S' '상품Z' '상품V' '상품O' '상품U' '상품L' '상품C' '상품I' '상품R' '상품X' '상품G'
 '상품P' '상품Q' '상품Y' '상품N' '상품W' '상품E' '상품K' '상품B' '상품F' '상품D' '상품M' '상품H'
 '상품T' '상품J']


### 금액 결측치 수정

In [11]:
#결측치 확인
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price         True
customer_name     False
purchase_data     False
purchase_month    False
dtype: bool

#결측치 수정
flg_is_null = uriage_data["item_price"].isnull()
for trg in list(uriage_data.loc[flg_is_null, "item_name"].unique()):
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"]==trg), "item_price"].max()
    uriage_data["item_price"].loc[(flg_is_null) & (uriage_data["item_name"]==trg)] = price
uriage_data.head()

In [13]:
import numpy as np

for trg in uriage_data.loc[flg_is_null, "item_name"].unique():
    price = uriage_data.loc[(~flg_is_null) & (uriage_data["item_name"]==trg), "item_price"].max()
    uriage_data.loc[(flg_is_null) & (uriage_data["item_name"]==trg), "item_price"] = price

uriage_data.head()


,purchase_date,item_name,item_price,customer_name,purchase_data,purchase_month
0,2019-06-13 18:02,상품A,100.0,김가온,2019-06-13 18:02:00,201906
1,2019-07-13 13:05,상품S,1900.0,김우찬,2019-07-13 13:05:00,201907
2,2019-05-11 19:42,상품A,100.0,김유찬,2019-05-11 19:42:00,201905
3,2019-02-12 23:40,상품Z,2600.0,김재현,2019-02-12 23:40:00,201902
4,2019-04-22 3:09,상품A,100.0,김강현,2019-04-22 03:09:00,201904


In [16]:
#위의 코드 한줄로 만들기
uriage_data["item_price"] = np.where(flg_is_null, uriage_data["item_price"].fillna(price), uriage_data["item_price"])
uriage_data.head()

,purchase_date,item_name,item_price,customer_name,purchase_data,purchase_month
0,2019-06-13 18:02,상품A,100.0,김가온,2019-06-13 18:02:00,201906
1,2019-07-13 13:05,상품S,1900.0,김우찬,2019-07-13 13:05:00,201907
2,2019-05-11 19:42,상품A,100.0,김유찬,2019-05-11 19:42:00,201905
3,2019-02-12 23:40,상품Z,2600.0,김재현,2019-02-12 23:40:00,201902
4,2019-04-22 3:09,상품A,100.0,김강현,2019-04-22 03:09:00,201904


In [18]:
#검증
uriage_data.isnull().any(axis=0)

purchase_date     False
item_name         False
item_price        False
customer_name     False
purchase_data     False
purchase_month    False
dtype: bool

In [21]:
for trg in list(uriage_data["item_price"].sort_values().unique()):
    print(trg + "의 최고가: " + str(uriage_data.loc[uriage_data["item_name"]==trg]
                               ["item_price"].max()) + "의 최저가: " + str(uriage_data.loc[uriage_data
                               ["item_name"]==trg]["item_price"].min(skipna=False)))

UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('float64'), dtype('<U7')) -> None

In [34]:
for trg in sorted(uriage_data["item_name"].unique()):
    trg_data = uriage_data[uriage_data["item_name"] == trg]
    print(str(trg) + "의 최고가: " + str(trg_data["item_price"].max()) + ", 최저가: " + str(trg_data["item_price"].min()))

상품A의 최고가: 100.0, 최저가: 100.0
상품B의 최고가: 200.0, 최저가: 200.0
상품C의 최고가: 300.0, 최저가: 300.0
상품D의 최고가: 400.0, 최저가: 400.0
상품E의 최고가: 500.0, 최저가: 500.0
상품F의 최고가: 600.0, 최저가: 600.0
상품G의 최고가: 700.0, 최저가: 700.0
상품H의 최고가: 800.0, 최저가: 800.0
상품I의 최고가: 900.0, 최저가: 900.0
상품J의 최고가: 1000.0, 최저가: 1000.0
상품K의 최고가: 1100.0, 최저가: 1100.0
상품L의 최고가: 1200.0, 최저가: 1200.0
상품M의 최고가: 1300.0, 최저가: 1300.0
상품N의 최고가: 1400.0, 최저가: 1400.0
상품O의 최고가: 1500.0, 최저가: 1500.0
상품P의 최고가: 1600.0, 최저가: 1600.0
상품Q의 최고가: 1700.0, 최저가: 1700.0
상품R의 최고가: 1800.0, 최저가: 1800.0
상품S의 최고가: 1900.0, 최저가: 1900.0
상품T의 최고가: 2000.0, 최저가: 2000.0
상품U의 최고가: 2100.0, 최저가: 2100.0
상품V의 최고가: 2200.0, 최저가: 2200.0
상품W의 최고가: 2300.0, 최저가: 2300.0
상품X의 최고가: 2400.0, 최저가: 2400.0
상품Y의 최고가: 2500.0, 최저가: 2500.0
상품Z의 최고가: 2600.0, 최저가: 2600.0


### 고객 이름 오류 수정

In [37]:
#고객정보 데이터 확인
kokyaku_data["고객이름"].head()

0    김 현성
1    김 도윤
2    김 지한
3    김 하윤
4    김 시온
Name: 고객이름, dtype: object

In [38]:
#매출이력 데이터 확인
uriage_data["customer_name"].head()

0    김가온
1    김우찬
2    김유찬
3    김재현
4    김강현
Name: customer_name, dtype: object

In [39]:
#고객정보 고객이름 칼럼의 공백 제거
kokyaku_data["고객이름"] = kokyaku_data["고객이름"].str.replace("  ","")
kokyaku_data["고객이름"] = kokyaku_data["고객이름"].str.replace(" ","")
kokyaku_data["고객이름"].head()

0    김현성
1    김도윤
2    김지한
3    김하윤
4    김시온
Name: 고객이름, dtype: object

### 날짜 오류 수정

In [43]:
#날짜 아닌 숫자로 읽히는 데이터 확인
flg_is_serial = kokyaku_data["등록일"].astype("str").str.isdigit()
flg_is_serial.sum()

22

In [44]:
fromSerial = pd.to_timedelta(kokyaku_data.loc[flg_is_serial, "등록일"].astype("float")
                            ,unit="D") + pd.to_datetime("1900/01/01")
fromSerial

1     2017-02-18
3     2017-05-19
4     2018-01-29
21    2017-07-06
27    2017-06-17
47    2017-01-08
49    2017-07-15
53    2017-04-10
76    2018-03-31
80    2018-01-12
99    2017-06-01
114   2018-06-05
118   2018-01-31
122   2018-04-18
139   2017-05-27
143   2017-03-26
155   2017-01-21
172   2018-03-24
179   2017-01-10
183   2017-07-26
186   2018-07-15
192   2018-06-10
Name: 등록일, dtype: datetime64[ns]

In [45]:
fromString = pd.to_datetime(kokyaku_data.loc[~flg_is_serial, "등록일"])
fromString

0     2018-01-04
2     2018-01-07
5     2017-06-20
6     2018-06-11
7     2017-05-19
         ...    
195   2017-06-20
196   2018-06-20
197   2017-04-29
198   2019-04-19
199   2019-04-23
Name: 등록일, Length: 178, dtype: datetime64[ns]

In [51]:
# 등록일 칼럼에 합치기
kokyaku_data["등록일"] = pd.concat([fromSerial, fromString])
kokyaku_data

,고객이름,지역,등록일,등록연월
0,김현성,H시,2018-01-04,201801
1,김도윤,E시,2017-02-18,201702
2,김지한,A시,2018-01-07,201801
3,김하윤,F시,2017-05-19,201705
4,김시온,E시,2018-01-29,201801
...,...,...,...,...
195,김재희,G시,2017-06-20,201706
196,김도영,E시,2018-06-20,201806
197,김이안,F시,2017-04-29,201704
198,김시현,H시,2019-04-19,201904


In [52]:
# 등록일에서 등록월 추출
kokyaku_data["등록연월"] = kokyaku_data["등록일"].dt.strftime("%Y%m")
rslt = kokyaku_data.groupby("등록연월").count()["고객이름"]
print(rslt)
print(len(kokyaku_data))

등록연월
201701    15
201702    11
201703    14
201704    15
201705    13
201706    14
201707    17
201801    13
201802    15
201803    17
201804     5
201805    19
201806    13
201807    17
201904     2
Name: 고객이름, dtype: int64
200


In [53]:
#숫자 항목 유무 재확인
flg_is_serial = kokyaku_data["등록일"].astype("str").str.isdigit()
flg_is_serial.sum()

0